In [1]:
from src.hopfield import *
from data.alphabet import alphabet_map
import itertools
import numpy as np
import random

In [2]:
def flatten_matrix(matrix):
  return np.array(matrix).flatten()

In [3]:
def generate_orthogonal_set(data):
  combinations = itertools.combinations(data.keys(), r=4)
  avg_dot_prod = []
  for g in combinations:
    group = np.array([ data[key] for key in g ])
    ortho_matrix = group.dot(group.T)
    np.fill_diagonal(ortho_matrix, 0)
    row, _ = ortho_matrix.shape
    avg_dot_prod.append((np.abs(ortho_matrix).sum() / (ortho_matrix.size - row), g))
  return avg_dot_prod

best = min(generate_orthogonal_set({ key: np.array([value]).flatten() for key, value in alphabet_map.items() }), key = lambda dot: dot[0])
worst = max(generate_orthogonal_set({ key: np.array([value]).flatten() for key, value in alphabet_map.items() }), key = lambda dot: dot[0])

In [4]:
alphabet_matrix = [ alphabet_map[key] for key in best[1] ]
flattened_train_set = [ flatten_matrix(alphmap) for alphmap in alphabet_matrix ]


# TODO evaluar tipo de ruido
# Evaluar funcion de energia
# 

In [61]:
import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots

# Create a NumPy array from your matrix
def print_letters(X: List, title = ""):
  fig = make_subplots(rows = 1, cols = len(X))
  colorscale = [[0, 'white'], [1, 'black']]

  for i, letter in enumerate(X):
    X_array = np.array(letter)
    fig.add_trace(go.Heatmap(z=X_array, colorscale=colorscale), row = 1, col = i + 1)
  fig.update_xaxes(showticklabels=False)
  fig.update_yaxes(showticklabels=False)
  fig.update_coloraxes(showscale=False)
  fig.update_layout(dict({ "title": title }))
  fig.show()

def print_list_progress(X: List[List], title = ""):
  fig = make_subplots(rows = len(X), cols = max([len(x) for x in X]))
  colorscale = [[0, 'white'], [1, 'black']]

  for i, row in enumerate(X):
    for j, letter in enumerate(row):
      X_array = np.array(letter)
      heatmap = go.Heatmap(z=X_array, colorscale=colorscale)
      fig.add_trace(heatmap, row = i + 1, col = j + 1)
      
  fig.update_xaxes(showticklabels=False)
  fig.update_yaxes(showticklabels=False)
  fig.update_coloraxes(showscale=False)
  fig.update_layout(dict({ "title": title, "width": 500 }))
  fig.show()

def print_energy_linegraph(x: List, title = ""):
  fig = go.Figure()
  fig.add_trace(go.Scatter(x = list(range(len(x))), y = x, mode='lines+markers'))
  fig.update_layout(dict({ "title": title }))
  fig.show()

print_letters([np.flipud(letter) for letter in alphabet_matrix], title = "Letras de mejor ortogonalidad")


In [59]:
def add_noise(matrix, percentage):
  m = np.copy(matrix)
  for i in range(len(matrix)):
    for j in range(len(matrix[0])):
      m[i][j] *= -1 if random.random() < percentage else 1
  return m

noisy_letters = [add_noise(letter, 0.40) for letter in alphabet_matrix]


In [65]:
hopfield_matrix = generate_hopfield_matrix(flattened_train_set)

print_letters([np.flipud(letter) for letter in noisy_letters], "Letras de consulta")

progresses = [ most_similar_pattern(hopfield_matrix, flatten_matrix(letter), 10) for letter in noisy_letters ]

print_list_progress([[np.flipud(np.reshape(letter[1], [5,5])) for letter in progress[3]] for progress in progresses], title = "Progreso de letra recuperada")

print_energy_linegraph([progress[2] for progress in progresses[1][3]], title = "Energía de recuperación")

temp = most_similar_pattern(hopfield_matrix, flatten_matrix(alphabet_map["I"]), 10)
